# Preprocessing Input Variants

This notebook processes variant data from multiple sources (ClinVar, COSMIC, and MAF) to create a unified dataset for downstream analysis in the UAVarPrior project.

In [ ]:
# Import required libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Load Dataset Files

First, we'll load the variant data from multiple sources:
1. ClinVar - Clinically relevant variants
2. COSMIC - Catalogue of Somatic Mutations in Cancer
3. MAF - Mutation Annotation Format (typically from TCGA)

In [ ]:
# Define paths to input data files
# You may need to adjust these paths to match your environment
data_dir = '../data'
clinvar_path = os.path.join(data_dir, 'clinvar_variants.csv')
cosmic_path = os.path.join(data_dir, 'cosmic_variants.csv')
maf_path = os.path.join(data_dir, 'tcga_maf_variants.csv')

# Load ClinVar data
print("Loading ClinVar data...")
cln_var_df = pd.read_csv(clinvar_path)
print(f"Loaded {len(cln_var_df)} ClinVar variants")
print(cln_var_df.head())
print("\nClinVar columns:", cln_var_df.columns.tolist())

# Load COSMIC data
print("\nLoading COSMIC data...")
cosmic_df = pd.read_csv(cosmic_path)
print(f"Loaded {len(cosmic_df)} COSMIC variants")
print(cosmic_df.head())
print("\nCOSMIC columns:", cosmic_df.columns.tolist())

# Load MAF data
print("\nLoading MAF data...")
df_maf = pd.read_csv(maf_path)
print(f"Loaded {len(df_maf)} MAF variants")
print(df_maf.head())
print("\nMAF columns:", df_maf.columns.tolist())

## Data Preprocessing

Each dataset may have different column names and formats. We'll examine the data and standardize the chromosome and position information.

In [ ]:
# Check for missing values in key columns
print("Missing values in key columns:")
print("\nClinVar:")
print(cln_var_df[['chrom', 'pos']].isna().sum())

print("\nCOSMIC:")
print(cosmic_df[['chrom', 'pos']].isna().sum())

print("\nMAF:")
print(df_maf[['chrom', 'pos']].isna().sum())

# Drop rows with missing chromosome or position information
cln_var_df = cln_var_df.dropna(subset=['chrom', 'pos'])
cosmic_df = cosmic_df.dropna(subset=['chrom', 'pos'])
df_maf = df_maf.dropna(subset=['chrom', 'pos'])

print("\nAfter dropping missing values:")
print(f"ClinVar: {len(cln_var_df)} variants")
print(f"COSMIC: {len(cosmic_df)} variants")
print(f"MAF: {len(df_maf)} variants")

## Standardize Chromosome Format

Ensure chromosome names are consistent across datasets (e.g., with or without "chr" prefix).

In [ ]:
# Function to standardize chromosome format
def standardize_chrom(chrom):
    """
    Standardize chromosome format to ensure consistency.
    Convert to string, remove 'chr' prefix if present, and handle special cases.
    """
    chrom = str(chrom)
    # Remove 'chr' prefix if present
    if chrom.startswith('chr'):
        chrom = chrom[3:]
    # Handle mitochondrial chromosome
    if chrom.lower() in ['m', 'mt']:
        chrom = 'MT'
    return chrom

# Apply standardization to chromosome columns
cln_var_df['chrom'] = cln_var_df['chrom'].apply(standardize_chrom)
cosmic_df['chrom'] = cosmic_df['chrom'].apply(standardize_chrom)
df_maf['chrom'] = df_maf['chrom'].apply(standardize_chrom)

# Check unique chromosome values to ensure consistency
print("Unique chromosome values after standardization:")
print("\nClinVar:", sorted(cln_var_df['chrom'].unique()))
print("\nCOSMIC:", sorted(cosmic_df['chrom'].unique()))
print("\nMAF:", sorted(df_maf['chrom'].unique()))

## Ensure Position is Integer Type

Make sure all position values are integers.

In [ ]:
# Convert position to integer type
cln_var_df['pos'] = cln_var_df['pos'].astype(int)
cosmic_df['pos'] = cosmic_df['pos'].astype(int)
df_maf['pos'] = df_maf['pos'].astype(int)

# Verify data types
print("Data types after conversion:")
print("\nClinVar:")
print(cln_var_df[['chrom', 'pos']].dtypes)
print("\nCOSMIC:")
print(cosmic_df[['chrom', 'pos']].dtypes)
print("\nMAF:")
print(df_maf[['chrom', 'pos']].dtypes)

## Combine Dataframes

Combine the three datasets, keeping only the chromosome and position columns, and remove duplicates.

In [ ]:
# Step 1: Extract Relevant Columns
# Extract only chrom and pos columns from each dataframe
cln_var_subset = cln_var_df[['chrom', 'pos']].copy()
cosmic_subset = cosmic_df[['chrom', 'pos']].copy()
df_maf_subset = df_maf[['chrom', 'pos']].copy()

# Step 2: Concatenate the Dataframes
# Combine all three dataframes
combined_df = pd.concat([cln_var_subset, cosmic_subset, df_maf_subset], axis=0)

# Step 3: Remove Duplicates
# Remove any duplicate rows (positions that appear in multiple datasets)
combined_df = combined_df.drop_duplicates()

# Step 4: Reset Index
# Reset the index to have a clean sequential index
combined_df = combined_df.reset_index(drop=True)

# Step 5: Verify and Standardize Data Types
# Ensure consistent data types across the combined dataframe
combined_df['chrom'] = combined_df['chrom'].astype(str)
combined_df['pos'] = combined_df['pos'].astype(int)

# Print summary information
print(f"Total variants after combining and removing duplicates: {len(combined_df)}")
print("\nSample of combined dataframe:")
print(combined_df.head())

# Print counts by source (before deduplication)
print(f"\nSource counts (before deduplication):")
print(f"ClinVar variants: {len(cln_var_subset)}")
print(f"COSMIC variants: {len(cosmic_subset)}")
print(f"MAF variants: {len(df_maf_subset)}")
print(f"Total combined (after deduplication): {len(combined_df)}")

## Distribution Analysis

Analyze the distribution of variants across chromosomes.

In [ ]:
# Count variants by chromosome
chrom_counts = combined_df['chrom'].value_counts().sort_index()

# Plot the distribution
plt.figure(figsize=(15, 6))
sns.barplot(x=chrom_counts.index, y=chrom_counts.values)
plt.title('Variant Distribution by Chromosome')
plt.xlabel('Chromosome')
plt.ylabel('Number of Variants')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Calculate percentage distribution
percentage_dist = (chrom_counts / chrom_counts.sum() * 100).round(2)
print("Percentage distribution by chromosome:")
for chrom, percentage in percentage_dist.items():
    print(f"Chromosome {chrom}: {percentage}%")

## Add Dummy Label Column

For compatibility with downstream analysis tools, add a dummy "label" column with zeros.

In [ ]:
# Add a label column (required by some downstream tools)
combined_df['label'] = 0

print("Added 'label' column with zeros:")
print(combined_df.head())

## Save the Processed Data

Save the combined and processed dataframe to a parquet file for efficient storage and access.

In [ ]:
# Create outputs directory if it doesn't exist
output_dir = os.path.join(os.path.dirname(os.path.abspath('__file__')), 'outputs')
os.makedirs(output_dir, exist_ok=True)
print(f"Saving large files to {output_dir} (not tracked by Git)")

# Save to parquet file with compression
output_path = os.path.join(output_dir, 'combined_variant_positions.parquet.gz')
combined_df.to_parquet(output_path, compression='gzip')
print(f"Saved combined variant positions to: {output_path}")

## Summary

This notebook has:
1. Loaded variant data from three sources (ClinVar, COSMIC, MAF)
2. Standardized chromosome and position information
3. Combined the datasets while removing duplicates
4. Added a dummy label column
5. Saved the processed data to a compressed parquet file

The resulting dataset can be used for downstream analysis in the UAVarPrior project.

In [13]:
import os
import re
import numpy as np
import pandas as pd
import gzip
print('Packages loaded')

Packages loaded


In [10]:
def read_vcf(vcf_file_path):
    """
    Reads data from vcf file and parse lines into pandas dataframe
        Parameters:
            vcf_file_path: str
        
        Returns:
            pandas dataframe with columns ["CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO"]
    """
    VCF_REQUIRED_COLS = ["#CHROM", "POS", "ID", "REF", "ALT"]
    with gzip.open(vcf_file_path, 'rb') as file_handle:
        lines = file_handle.readlines()
        
        # handling first few lines
        index = 0
        for index, line in enumerate(lines):
            line = line.decode('utf8')
            if '#' not in line:
                break
            if "#CHROM" in line:
                cols = line.strip().split('\t')
                if cols[:5] != VCF_REQUIRED_COLS:
                    raise ValueError(
                        "First 5 columns in file {0} were {1}. "
                        "Expected columns: {2}".format(
                            input_path, cols[:5], VCF_REQUIRED_COLS))
                index += 1
                break  
         
    # handling remaining lines of vcf file
    variants = []
    for line in lines[index:]:
        line = line.decode('utf8')
        cols = line.strip().split('\t')
        variants.append(cols)

    df = pd.DataFrame(variants)
    print(df.shape)
    df.columns = ["CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO"]
    return df

def remove_extra_chroms(df):
    chroms = [str(chrom) for chrom in range(1, 23)]
    df_var = pd.DataFrame()
    for chrom in chroms:
        chr_df = df[df['CHROM']==str(chrom)]
#         print(chrom, chr_df.shape)
        df_var = pd.concat([df_var, chr_df])
    return df_var


def get_unique_id_dict(df_var):
    """
    Extracts unique ID types from info column of vcf data
        Parameters:
            df_var: pandas dataframe with info as one of the columns
        
        Returns:
            dictionary of unique ids with None value as default
    """
    max_info = 0
    unique_ids = []
    for info_row in df_var['INFO'].values:
        splits = info_row.split(';')
        slen = len(splits)
        if max_info <= slen:
            max_info = slen
            for splt in splits:
                unique_ids.append(splt.split('=')[0])

    unique_ids = np.unique(np.array(unique_ids))
    uniq_id_dict = {'DBVARID': None} # Somehow this key couldn't be extracted by above code
    for uid in unique_ids:
        uniq_id_dict[uid] = None
    return uniq_id_dict

def info_row_expand(info_row, uniq_id_dict):
    """
    Expands info row by splitting it into different ids
        Parameters:
            info_row (str): info text row
            uniq_id_dict (dictionary): dictionary with unique ids
            
        Returns:
            dictionary with unique ids as keys and values extracted from info as values
    """
    temp_dict = {}
    for key, value in uniq_id_dict.items():
        temp_dict[key] = value
    keys = uniq_id_dict.keys()
        
    splits = info_row.split(';')
    for spl in splits:
        temp = spl.split('=')
        if temp[0] in keys:
            temp_dict[temp[0]] = temp[1]
        else:
            print(f'Key: {temp[0]} value: {temp[1]} are extra items')
    return np.array(list(temp_dict.values()))

In [14]:
clin_var_file = '/scratch/ml-csm/projects/fgenom/gve/data/human/clinVar/clinvar_20250202.vcf.gz'
df = read_vcf(clin_var_file)


df = remove_extra_chroms(df)

uniq_id_dict = get_unique_id_dict(df)
print(len(uniq_id_dict))


values = []
for info_row in df['INFO'].values:
    row_values = info_row_expand(info_row, uniq_id_dict)
    values.append(row_values)
    
values = np.array(values)
cln_var_df = pd.DataFrame(values)
cln_var_df.columns = list(uniq_id_dict.keys())
cln_var_df = pd.concat([df.iloc[:, [0, 1]], cln_var_df.loc[:, ['CLNDN']]], axis=1)
# cln_var_df = pd.concat([df.iloc[:, [0, 1]], cln_var_df], axis=1)
del df
del values
cln_var_df

(3221834, 8)
35


,CHROM,POS,CLNDN
0,1,66926,Retinitis_pigmentosa
1,1,69134,not_specified
2,1,69314,not_specified
3,1,69423,not_specified
4,1,69581,not_specified
...,...,...,...
3099325,22,50777972,not_specified
3099326,22,50777976,not_specified
3099327,22,50782204,not_specified
3099328,22,50782243,not_specified


In [19]:
cln_var_df.columns = ['chrom', 'pos', 'CLNDN']
cln_var_df['chrom'] = cln_var_df['chrom'].astype(np.int8)
cln_var_df['pos'] = cln_var_df['pos'].astype(np.uint32)
cln_var_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3099330 entries, 0 to 3099329
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   chrom   int8  
 1   pos     uint32
 2   CLNDN   object
dtypes: int8(1), object(1), uint32(1)
memory usage: 62.1+ MB


In [15]:
# Load COSMIC noncoding variant data and convert column types
cosmic_df = pd.read_parquet(
    '/scratch/ml-csm/projects/fgenom/gve/data/human/COSMIC/COSMIC_noncoding_var.parquet.gzip',
    columns=['chrom', 'pos']
)
cosmic_df['chrom'] = cosmic_df['chrom'].astype(np.int8)
cosmic_df['pos'] = cosmic_df['pos'].astype(np.uint32)
cosmic_df

,chrom,pos
0,1,10108
1,1,10108
2,1,10108
3,1,10151
4,1,10151
...,...,...
16727025,9,138258488
16727026,9,138258493
16727027,9,138258852
16727028,9,138258875


In [17]:
inPath = '/scratch/ml-csm/datasets/genomics/ref-genome/human/GRCh38/ensembl/variants/processed/'
df_maf = pd.read_parquet(inPath+'1000GENOMES-release114-maf.parquet.gz')
df_maf

,chrom,pos,id,ref,alt,maf,category
0,1,51479,rs116400033,T,A,0.104199,common
1,1,51954,rs185832753,G,"A,C,T",0.000196,rare
2,1,54490,rs141149254,G,"A,C",0.096939,common
3,1,54669,rs532505601,C,T,0.000196,rare
4,1,54716,rs569128616,C,"A,T",0.213305,common
...,...,...,...,...,...,...,...
69542010,22,50795771,rs6010092,T,C,0.032182,none
69542011,22,50795915,rs374867791,G,T,0.076727,common
69542012,22,50796090,rs537052521,C,G,0.000196,rare
69542013,22,50796204,rs6010093,G,"A,C",0.002943,none


In [21]:
df_maf['chrom'] = df_maf['chrom'].astype(np.int8)
df_maf['pos'] = df_maf['pos'].astype(np.uint32)
df_maf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69542015 entries, 0 to 69542014
Data columns (total 7 columns):
 #   Column    Dtype  
---  ------    -----  
 0   chrom     int8   
 1   pos       uint32 
 2   id        object 
 3   ref       object 
 4   alt       object 
 5   maf       float32
 6   category  object 
dtypes: float32(1), int8(1), object(4), uint32(1)
memory usage: 2.7+ GB


In [28]:
# Combine all three dataframes, keeping only chrom and pos columns

# Extract only the relevant columns from each dataframe
cln_var_subset = cln_var_df[['chrom', 'pos']]
cosmic_subset = cosmic_df[['chrom', 'pos']]
df_maf_subset = df_maf[['chrom', 'pos']]

# Concatenate all dataframes
combined_df = pd.concat([cln_var_subset, cosmic_subset, df_maf_subset], axis=0)

# Remove duplicates
combined_df = combined_df.drop_duplicates()

# Reset index
combined_df = combined_df.reset_index(drop=True)

# Add a dummy 'label' column with zeros
# combined_df['label'] = 0

# Display information about the combined dataframe
print(f"Original dataframe sizes:")
print(f"ClinVar: {cln_var_subset.shape[0]:,} rows")
print(f"COSMIC: {cosmic_subset.shape[0]:,} rows")
print(f"MAF: {df_maf_subset.shape[0]:,} rows")
print(f"Total rows before deduplication: {cln_var_subset.shape[0] + cosmic_subset.shape[0] + df_maf_subset.shape[0]:,}")
print(f"Combined dataframe: {combined_df.shape[0]:,} unique rows with {combined_df.shape[1]} columns")

# Display the first few rows of the combined dataframe
combined_df.head()

Original dataframe sizes:
ClinVar: 3,099,330 rows
COSMIC: 16,727,030 rows
MAF: 69,542,015 rows
Total rows before deduplication: 89,368,375
Combined dataframe: 77,304,080 unique rows with 2 columns


,chrom,pos
0,1,66926
1,1,69134
2,1,69314
3,1,69423
4,1,69581


In [29]:
# Check data types
# combined_df.info()

# Ensure data types are consistent
combined_df['chrom'] = combined_df['chrom'].astype(np.int8)
combined_df['pos'] = combined_df['pos'].astype(np.uint32)
# combined_df['label'] = combined_df['label'].astype(np.int8)  # Use int8 for the label column to save memory

# # Save the combined dataframe to parquet format
# output_path = '/scratch/ml-csm/projects/fgenom/gve/data/combined_variant_positions.parquet.gz'
# combined_df.to_parquet(output_path, compression='gzip')
# print(f"Combined variant positions saved to: {output_path}")

# Memory usage information
memory_usage = combined_df.memory_usage(deep=True).sum() / (1024 * 1024)
print(f"Memory usage of combined dataframe: {memory_usage:.2f} MB")

Memory usage of combined dataframe: 368.61 MB


In [27]:
output_path = '/scratch/ml-csm/projects/fgenom/gve/data/combined_variant_positions.tsv'
combined_df.to_csv(output_path, sep='\t', header=None, index=False)
combined_df.shape

(77304080, 3)

In [30]:
chr_vcf_folder = '/scratch/ml-csm/projects/fgenom/gve/data/human/ensembl/GRCh38/variant/processed/chroms/'
vcf_files = os.listdir(chr_vcf_folder)

vcf_files.remove('homo_sapiens-chrX.tsv')
vcf_files.remove('homo_sapiens-chrY.tsv')
vcf_files.sort()
print(f'chromosome var vcf files: {len(vcf_files)}')

chromosome var vcf files: 22


In [38]:
top_vcf = pd.DataFrame()

for vcf_file in vcf_files:
    # Read the VCF file
    vcf = pd.read_csv(chr_vcf_folder+vcf_file, sep='\t')
    print(f'Original VCF file shape: {vcf.shape}')
    
    # Extract chromosome number from filename
    chrom = int(re.search(r'chr(\d+)', vcf_file).group(1))
    
    # Convert VCF column types to match combined_df for proper comparison
    vcf['#CHROM'] = vcf['#CHROM'].astype(np.int8)
    vcf['POS'] = vcf['POS'].astype(np.uint32)
    
    # Filter VCF based on chromosome and position from combined_df
    # First get positions from combined_df for the current chromosome
    chrom_positions = combined_df[combined_df['chrom'] == chrom]['pos'].values
    
    # Then filter VCF to only include rows with positions in chrom_positions
    chr_top_vcf = vcf[vcf['POS'].isin(chrom_positions)]
    
    # Concatenate with existing top_vcf dataframe
    top_vcf = pd.concat([top_vcf, chr_top_vcf])
    
    print(f'{vcf_file}: filtered to {chr_top_vcf.shape} variants based on position matching')

# Remove duplicates based on variant ID
dup_index = top_vcf.ID.duplicated()
top_vcf = top_vcf[~dup_index]
print(f'Final top_vcf after removing duplicates: {top_vcf.shape}')

# Display the first few rows of the filtered VCF data
top_vcf.head()

Original VCF file shape: (54950388, 6)
homo_sapiens-chr1.tsv: filtered to (6335460, 6) variants based on position matching
Original VCF file shape: (32448977, 6)
homo_sapiens-chr10.tsv: filtered to (3866487, 6) variants based on position matching
Original VCF file shape: (33255335, 6)
homo_sapiens-chr11.tsv: filtered to (4023851, 6) variants based on position matching
Original VCF file shape: (32146515, 6)
homo_sapiens-chr12.tsv: filtered to (3752760, 6) variants based on position matching
Original VCF file shape: (23665483, 6)
homo_sapiens-chr13.tsv: filtered to (2811679, 6) variants based on position matching
Original VCF file shape: (21621402, 6)
homo_sapiens-chr14.tsv: filtered to (2624409, 6) variants based on position matching
Original VCF file shape: (20235594, 6)
homo_sapiens-chr15.tsv: filtered to (2363241, 6) variants based on position matching
Original VCF file shape: (22226709, 6)
homo_sapiens-chr16.tsv: filtered to (2675267, 6) variants based on position matching
Original 

,#CHROM,POS,ID,REF,ALT,STRAND
34,1,10108,rs62651026,C,T,.
63,1,10151,rs1570391830,T,A,.
86,1,10175,rs1557426757,TAACC,T,.
90,1,10179,rs1312716213,CTAA,C,.
91,1,10179,rs1557426763,CTAAC,C,.


In [39]:
top_vcf.to_csv('/scratch/ml-csm/projects/fgenom/gve/data/comb_clinVar_COSMIC_MAF.tsv', sep='\t', header=None, index=False)